In [71]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors
from surprise import SVD
import implicit

In [91]:
df = pd.read_csv('/Users/aaronfalloon/Downloads/2ea54c57-868c-41f3-a397-18adac14ef15.csv')

In [92]:
df.columns = ['service', 'contractor']

In [93]:
df = df.query('contractor != "[null]"')

In [94]:
services = []
contractors = []

In [95]:
for row in df.itertuples(index=False):
    row_services = row[0][1: -1].split(',')
    row_contractors = row[1][1: -1].split(',')
    if len(row_services) > 1 and len(row_services) != len(row_contractors):
        continue
    if len(row_services) == 1:
        services += (row_services * len(row_contractors))
    else:
        services += row_services
    contractors += row_contractors

In [96]:
final = pd.DataFrame()
final = final.assign(service=services)
final = final.assign(contractor=contractors)

In [97]:
final.iloc[:10, :]

,service,contractor
0,Timber harvesting services,S.C. Narilserv S.R.L.
1,Refuse-collection vehicles,Moteco Service S.r.l.
2,Urban furniture,Ondelia
3,Pharmaceutical products,Janssen-Cilag Polska Sp. z o.o.
4,Performance review services,Wiesława Fabian
5,Performance review services,Izabela Maria Małyszko
6,Performance review services,Indywidualna Specjalistyczna Praktyka Lekarsk...
7,Performance review services,Jolanta Foltman
8,Performance review services,Mariusz Domański Prywatna Praktyka Lekarska
9,Performance review services,Agnieszka Jankowska-Zduńczyk


In [98]:
services_unique = final['service'].unique()

In [99]:
services_map = dict(zip(services_unique, range(0, len(services_unique))))

In [100]:
contractors_unique = final['contractor'].unique()

In [101]:
contractors_map = dict(zip(contractors_unique, range(0, len(contractors_unique))))

In [102]:
service_by_contractor = np.zeros((len(services_unique), len(contractors_unique)))

In [103]:
for row in final.itertuples(index=False):
    service_by_contractor[services_map[row[0]], contractors_map[row[1]]] = 1

In [105]:
service_by_contractor_sparse = coo_matrix(service_by_contractor)

In [106]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [107]:
model.fit(service_by_contractor_sparse)

100%|██████████| 15.0/15 [01:20<00:00,  4.51s/it]


In [118]:
user_items = service_by_contractor_sparse.T.tocsr()

In [138]:
user_id = 9091
print(contractors_unique[user_id])
recs = model.recommend(user_id, user_items)
indices = [rec[0] for rec in recs]
[(services_unique[rec[0]], rec[1]) for rec in recs]

Helder


[('Cleaning and sanitation services', 0.06214285),
 ('Office cleaning services', 0.016611602),
 ('School cleaning services', 0.007089855),
 ('Horticultural services', 0.0055942712),
 ('Translation services', 0.0055882726),
 ('Accommodation', 0.004304366),
 ('Timber harvesting services', 0.0038818605),
 ('Taxi services', 0.0035873428),
 ('Network equipment', 0.003251581),
 ('Orthopaedic prostheses', 0.003213107)]